# Text Classification Using Keras & TensorFlow on Amazon SageMaker

Full lab guide can be found here: https://github.com/aws-samples/amazon-sagemaker-keras-text-classification

In [1]:
%cd data
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip && unzip NewsAggregatorDataset.zip
!wget http://nlp.stanford.edu/data/glove.6B.zip && unzip glove.6B.zip
!rm 2pageSessions.csv glove.6B.200d.txt glove.6B.50d.txt glove.6B.300d.txt glove.6B.zip readme.txt NewsAggregatorDataset.zip && rm -rf __MACOSX/    

/home/ec2-user/SageMaker/amazon-sagemaker-keras-text-classification/data
--2019-06-11 07:06:43--  https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29224203 (28M) [application/x-httpd-php]
Saving to: ‘NewsAggregatorDataset.zip’

NewsAggregatorDatas 100%[===================>]  27.87M  14.2MB/s    in 2.0s    

2019-06-11 07:06:45 (14.2 MB/s) - ‘NewsAggregatorDataset.zip’ saved [29224203/29224203]

Archive:  NewsAggregatorDataset.zip
  inflating: 2pageSessions.csv       
   creating: __MACOSX/
  inflating: __MACOSX/._2pageSessions.csv  
  inflating: newsCorpora.csv         
  inflating: __MACOSX/._newsCorpora.csv  
  inflating: readme.txt              
  inflating: __MACOSX/._readme.txt   
--2019-06-11 07:06:46--  http://nlp.stanford.edu/

# Lab 1: Data Exploration

In [2]:
import pandas as pd
import tensorflow as tf
import re
import numpy as np
import os

from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.utils import to_categorical

In [3]:
column_names = ["TITLE", "URL", "PUBLISHER", "CATEGORY", "STORY", "HOSTNAME", "TIMESTAMP"]
news_dataset = pd.read_csv(os.path.join('.', 'newsCorpora.csv'), names=column_names, header=None, delimiter='\t')
news_dataset.head()

,TITLE,URL,PUBLISHER,CATEGORY,STORY,HOSTNAME,TIMESTAMP
1,"Fed official says weak data caused by weather,...",http://www.latimes.com/business/money/la-fi-mo...,Los Angeles Times,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.latimes.com,1394470370698
2,Fed's Charles Plosser sees high bar for change...,http://www.livemint.com/Politics/H2EvwJSK2VE6O...,Livemint,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.livemint.com,1394470371207
3,US open: Stocks fall after Fed official hints ...,http://www.ifamagazine.com/news/us-open-stocks...,IFA Magazine,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,1394470371550
4,"Fed risks falling 'behind the curve', Charles ...",http://www.ifamagazine.com/news/fed-risks-fall...,IFA Magazine,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,1394470371793
5,Fed's Plosser: Nasty Weather Has Curbed Job Gr...,http://www.moneynews.com/Economy/federal-reser...,Moneynews,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.moneynews.com,1394470372027


In [4]:
news_dataset_sampled = news_dataset.sample(frac=0.00005)
for i, n in enumerate(range(news_dataset_sampled.shape[0])):    
    category = news_dataset_sampled.iloc[i][3]
    if category == "b":
        category = "Business"
    elif category == "t":
        category = "Science & Technology"
    elif category == "e":
        category = "Entertainment"
    elif category == "m":
        category = "Health & Medicine"
    else:
        category = "unknown"
    print("{}. {} - {}".format(n+1, news_dataset_sampled.iloc[i][0], category))

1. Progress on job growth: Really! - Business
2. Strike a Pose! Kim Kardashian Finally Lands Vogue Magazine Cover! - Entertainment
3. Fargo: “Morton's Fork” - Entertainment
4. See the 7-Carat Engagement Ring George Clooney Gave Amal Alamuddin! - Entertainment
5. Will Smith may star in NFL concussion movie 'Game Brain' - Entertainment
6. Yandex to buy car classified website for $175m - Science & Technology
7. Moderate losses for bullions - Business
8. Facebook raises revenue from mobile ads - Business
9. Report: Morgan Stanley eyes compensation cuts - Business
10. US auto sales climb in March - Business
11. Men's Wearhouse, Jos. A. Bank merger elicits caution from financial analysts  ... - Business
12. Rick Case, AutoNation among new-vehicle dealers innovating to boost used-car  ... - Business
13. Here's how your iPhone just got a whole lot better - Science & Technology
14. Dropbox Buys Photo Stream Alternative 'Loom' For Its New Carousel App - Science & Technology
15. McDonald's offers

![STOP HERE](images/sm-keras-stop-sign.jpg)

## Stop here. Please switch back to the [lab guide](https://github.com/aws-samples/amazon-sagemaker-keras-text-classification) on Github and complete labs 2 and 3. You will come back to this notebook later in Lab 4.




# Lab 2: Building the SageMaker TensorFlow Container

In [5]:
%cd ~
!git clone https://github.com/aws/sagemaker-tensorflow-container.git
%cd sagemaker-tensorflow-container/docker/1.8.0/base
!docker build -t tensorflow-base:1.8.0-cpu-py2 -f Dockerfile.cpu .
!docker images

/home/ec2-user
fatal: destination path 'sagemaker-tensorflow-container' already exists and is not an empty directory.
/home/ec2-user/sagemaker-tensorflow-container/docker/1.8.0/base
Sending build context to Docker daemon  8.704kB
Step 1/9 : FROM ubuntu:16.04
16.04: Pulling from library/ubuntu

e2e5f967: Pulling fs layer 
6638ac9f: Pulling fs layer 
7d6d954b: Pulling fs layer 
Digest: sha256:cad5e101ab30bb7f7698b277dd49090f520fe063335643990ce8fbd15ff920ef
Status: Downloaded newer image for ubuntu:16.04
 ---> 2a697363a870
Step 2/9 : RUN apt-get update && apt-get install -y --no-install-recommends         build-essential         curl         git         libcurl3-dev         libfreetype6-dev         libpng12-dev         libzmq3-dev         pkg-config         python-dev         rsync         software-properties-common         unzip         zip         zlib1g-dev         openjdk-8-jdk         openjdk-8-jre-headless         wget         vim         iputils-ping         nginx         &&     ap

0 upgraded, 239 newly installed, 0 to remove and 15 not upgraded.
Need to get 183 MB of archives.
After this operation, 801 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu xenial/main amd64 libjson-c2 amd64 0.11-4ubuntu2 [22.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu xenial/main amd64 libpopt0 amd64 1.16-10 [26.0 kB]
Get:3 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libssl1.0.0 amd64 1.0.2g-1ubuntu4.15 [1084 kB]
Get:4 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libpython3.5-minimal amd64 3.5.2-2ubuntu0~16.04.5 [524 kB]
Get:5 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libexpat1 amd64 2.1.0-7ubuntu0.16.04.3 [71.2 kB]
Get:6 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 python3.5-minimal amd64 3.5.2-2ubuntu0~16.04.5 [1598 kB]
Get:7 http://archive.ubuntu.com/ubuntu xenial/main amd64 python3-minimal amd64 3.5.1-3 [23.3 kB]
Get:8 http://archive.ubuntu.com/ubuntu xenial/main amd64 mime-support all 3.5

Get:78 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libroken18-heimdal amd64 1.7~git20150920+dfsg-4ubuntu1.16.04.1 [41.4 kB]
Get:79 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libasn1-8-heimdal amd64 1.7~git20150920+dfsg-4ubuntu1.16.04.1 [174 kB]
Get:80 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libkrb5support0 amd64 1.13.2+dfsg-5ubuntu2.1 [31.2 kB]
Get:81 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libk5crypto3 amd64 1.13.2+dfsg-5ubuntu2.1 [81.3 kB]
Get:82 http://archive.ubuntu.com/ubuntu xenial/main amd64 libkeyutils1 amd64 1.5.9-8ubuntu1 [9904 B]
Get:83 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libkrb5-3 amd64 1.13.2+dfsg-5ubuntu2.1 [273 kB]
Get:84 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libgssapi-krb5-2 amd64 1.13.2+dfsg-5ubuntu2.1 [120 kB]
Get:85 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libhcrypto4-heimdal amd64 1.7~git20150920+dfsg-4ubuntu1.16.04.1 [85.0 kB]
Get:86 ht

Get:157 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 ca-certificates-java all 20160321ubuntu1 [12.5 kB]
Get:158 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 curl amd64 7.47.0-1ubuntu2.13 [139 kB]
Get:159 http://archive.ubuntu.com/ubuntu xenial/main amd64 liberror-perl all 0.17-1.2 [19.6 kB]
Get:160 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 git-man all 1:2.7.4-0ubuntu1.6 [736 kB]
Get:161 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 git amd64 1:2.7.4-0ubuntu1.6 [3176 kB]
Get:162 http://archive.ubuntu.com/ubuntu xenial/main amd64 libasound2-data all 1.1.0-0ubuntu1 [29.4 kB]
Get:163 http://archive.ubuntu.com/ubuntu xenial/main amd64 libasound2 amd64 1.1.0-0ubuntu1 [350 kB]
Get:164 http://archive.ubuntu.com/ubuntu xenial/main amd64 libatk1.0-data all 2.18.0-1 [17.1 kB]
Get:165 http://archive.ubuntu.com/ubuntu xenial/main amd64 libatk1.0-0 amd64 2.18.0-1 [56.9 kB]
Get:166 http://archive.ubuntu.com/ubuntu xenial/main amd64 libpixman-1

Get:238 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 vim amd64 2:7.4.1689-3ubuntu1.2 [1036 kB]
Get:239 http://archive.ubuntu.com/ubuntu xenial/main amd64 zip amd64 3.0-11 [158 kB]
debconf: delaying package configuration, since apt-utils is not installed
Fetched 183 MB in 5s (36.0 MB/s)
Selecting previously unselected package libjson-c2:amd64.
(Reading database ... 4777 files and directories currently installed.)
Preparing to unpack .../libjson-c2_0.11-4ubuntu2_amd64.deb ...
Unpacking libjson-c2:amd64 (0.11-4ubuntu2) ...
Selecting previously unselected package libpopt0:amd64.
Preparing to unpack .../libpopt0_1.16-10_amd64.deb ...
Unpacking libpopt0:amd64 (1.16-10) ...
Selecting previously unselected package libssl1.0.0:amd64.
Preparing to unpack .../libssl1.0.0_1.0.2g-1ubuntu4.15_amd64.deb ...
Unpacking libssl1.0.0:amd64 (1.0.2g-1ubuntu4.15) ...
Selecting previously unselected package libpython3.5-minimal:amd64.
Preparing to unpack .../libpython3.5-minimal_3.5.2-2ubuntu0~1

...

 69650K .......... .......... .......... .......... .......... 77% 6.63M 0s
 69700K .......... .......... .......... .......... .......... 77%  314M 0s
 69750K .......... .......... .......... .......... .......... 77%  322M 0s
 69800K .......... .......... .......... .......... .......... 77%  336M 0s
 69850K .......... .......... .......... .......... .......... 77%  311M 0s
 69900K .......... .......... .......... .......... .......... 77%  334M 0s
 69950K .......... .......... .......... .......... .......... 77%  331M 0s
 70000K .......... .......... .......... .......... .......... 77%  341M 0s
 70050K .......... .......... .......... .......... .......... 78%  254M 0s
 70100K .......... .......... .......... .......... .......... 78%  310M 0s
 70150K .......... .......... .......... .......... .......... 78% 20.8M 0s
 70200K .......... .......... .......... .......... .......... 78%  320M 0s
 70250K .......... .......... .......... .......... .......... 78%  292M 0s
 70300K ....

 73350K .......... .......... .......... .......... .......... 81% 17.7M 0s
 73400K .......... .......... .......... .......... .......... 81%  285M 0s
 73450K .......... .......... .......... .......... .......... 81%  280M 0s
 73500K .......... .......... .......... .......... .......... 81%  348M 0s
 73550K .......... .......... .......... .......... .......... 81%  372M 0s
 73600K .......... .......... .......... .......... .......... 81%  325M 0s
 73650K .......... .......... .......... .......... .......... 82%  301M 0s
 73700K .......... .......... .......... .......... .......... 82%  367M 0s
 73750K .......... .......... .......... .......... .......... 82%  340M 0s
 73800K .......... .......... .......... .......... .......... 82% 14.2M 0s
 73850K .......... .......... .......... .......... .......... 82%  291M 0s
 73900K .......... .......... .......... .......... .......... 82%  313M 0s
 73950K .......... .......... .......... .......... .......... 82%  258M 0s
 74000K ....

 80250K .......... .......... .......... .......... .......... 89% 65.3M 0s
 80300K .......... .......... .......... .......... .......... 89%  158M 0s
 80350K .......... .......... .......... .......... .......... 89%  112M 0s
 80400K .......... .......... .......... .......... .......... 89%  329M 0s
 80450K .......... .......... .......... .......... .......... 89% 10.1M 0s
 80500K .......... .......... .......... .......... .......... 89%  223M 0s
 80550K .......... .......... .......... .......... .......... 89%  259M 0s
 80600K .......... .......... .......... .......... .......... 89%  295M 0s
 80650K .......... .......... .......... .......... .......... 89%  304M 0s
 80700K .......... .......... .......... .......... .......... 89%  297M 0s
 80750K .......... .......... .......... .......... .......... 89%  262M 0s
 80800K .......... .......... .......... .......... .......... 89%  225M 0s
 80850K .......... .......... .......... .......... .......... 90%  292M 0s
 80900K ....

 83650K .......... .......... .......... .......... .......... 93% 76.4M 0s
 83700K .......... .......... .......... .......... .......... 93% 90.7M 0s
 83750K .......... .......... .......... .......... .......... 93%  249M 0s
 83800K .......... .......... .......... .......... .......... 93%  176M 0s
 83850K .......... .......... .......... .......... .......... 93%  181M 0s
 83900K .......... .......... .......... .......... .......... 93% 17.2M 0s
 83950K .......... .......... .......... .......... .......... 93%  220M 0s
 84000K .......... .......... .......... .......... .......... 93%  331M 0s
 84050K .......... .......... .......... .......... .......... 93%  342M 0s
 84100K .......... .......... .......... .......... .......... 93%  312M 0s
 84150K .......... .......... .......... .......... .......... 93%  259M 0s
 84200K .......... .......... .......... .......... .......... 93%  323M 0s
 84250K .......... .......... .......... .......... .......... 93%  368M 0s
 84300K ....

Selecting previously unselected package tensorflow-model-server.
(Reading database ... 17711 files and directories currently installed.)
Preparing to unpack tensorflow-model-server_1.7.0_all.deb ...
Unpacking tensorflow-model-server (1.7.0) ...
Setting up tensorflow-model-server (1.7.0) ...
Removing intermediate container f3fa2fcd533c
 ---> 8099d629a3f4
Step 9/9 : RUN add-apt-repository ppa:ubuntu-toolchain-r/test -y &&     apt-get update &&     apt-get install -y libstdc++6
 ---> Running in fd23f7859b63
gpg: keyring `/tmp/tmpavas8iqb/secring.gpg' created
gpg: keyring `/tmp/tmpavas8iqb/pubring.gpg' created
gpg: requesting key BA9EF27F from hkp server keyserver.ubuntu.com
gpg: /tmp/tmpavas8iqb/trustdb.gpg: trustdb created
gpg: key BA9EF27F: public key "Launchpad Toolchain builds" imported
gpg: Total number processed: 1
gpg:               imported: 1  (RSA: 1)
OK
Get:1 http://security.ubuntu.com/ubuntu xenial-security InRelease [109 kB]
Get:2 http://archive.ubuntu.com/ubuntu xenial InRel

In [35]:
%cd ~/SageMaker/amazon-sagemaker-keras-text-classification/container/

/home/ec2-user/SageMaker/amazon-sagemaker-keras-text-classification/container


In [7]:
%%writefile Dockerfile
# Build an image that can do training and inference in SageMaker

FROM tensorflow-base:1.8.0-cpu-py2

ENV PATH="/opt/program:${PATH}"

# Set up the program in the image
COPY sagemaker_keras_text_classification /opt/program
WORKDIR /opt/program

Writing Dockerfile


In [36]:
!docker build -t sagemaker-keras-text-class:latest .

Sending build context to Docker daemon  459.7MB
Step 1/4 : FROM tensorflow-base:1.8.0-cpu-py2
 ---> f0bfaa074d3e
Step 2/4 : ENV PATH="/opt/program:${PATH}"
 ---> Using cache
 ---> c48badb6665c
Step 3/4 : COPY sagemaker_keras_text_classification /opt/program
 ---> b45e426f17c2
Step 4/4 : WORKDIR /opt/program
 ---> Running in 296900a0392c
Removing intermediate container 296900a0392c
 ---> 31e4ff58ddce
Successfully built 31e4ff58ddce
Successfully tagged sagemaker-keras-text-class:latest


In [37]:
!docker images

REPOSITORY                                                                         TAG                 IMAGE ID            CREATED             SIZE
sagemaker-keras-text-class                                                         latest              31e4ff58ddce        2 seconds ago       1.91GB
741855114961.dkr.ecr.us-east-1.amazonaws.com/sagemaker-keras-text-classification   latest              8e715bc36ca2        2 hours ago         1.91GB
sagemaker-keras-text-classification                                                latest              8e715bc36ca2        2 hours ago         1.91GB
tensorflow-base                                                                    1.8.0-cpu-py2       f0bfaa074d3e        2 hours ago         1.91GB
ubuntu                                                                             16.04               2a697363a870        3 weeks ago         119MB


# Lab 3: Local Testing of Training & Inference Code

In [10]:
%cd ~/SageMaker/amazon-sagemaker-keras-text-classification/data
!cp -a . ../container/local_test/test_dir/input/data/training/

/home/ec2-user/SageMaker/amazon-sagemaker-keras-text-classification/data


In [38]:
%cd ~/SageMaker/amazon-sagemaker-keras-text-classification/container/local_test
!./train_local.sh sagemaker-keras-text-class:latest

/home/ec2-user/SageMaker/amazon-sagemaker-keras-text-classification/container/local_test
Instructions for updating:
Colocations handled automatically by placer.
2019-06-11 08:52:04.531505: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2019-06-11 08:52:04.537304: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2400070000 Hz
2019-06-11 08:52:04.537483: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0xb95aaf0 executing computations on platform Host. Devices:
2019-06-11 08:52:04.537504: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
Instructions for updating:
Use tf.cast instead.
Starting the training.
                                               TITLE  ...      TIMESTAMP
1  Fed official says weak data caused by weather,...  ...  1394470370698
2  Fed's Charles Plosser sees high bar for change..

In [ ]:
!./serve_local.sh sagemaker-keras-text-class:latest

In [48]:
%cd ~/SageMaker/amazon-sagemaker-keras-text-classification/container/local_test/
!./predict.sh input.json application/json

/home/ec2-user/SageMaker/amazon-sagemaker-keras-text-classification/container/local_test
curl: (7) Failed to connect to localhost port 8080: Connection refused


# Lab 4: Training and Hosting your Algorithm in Amazon SageMaker


### Building and registering the container

The following shell code shows how to build the container image using `docker build` and push the container image to ECR using `docker push`. 

This code looks for an ECR repository in the account you're using and the current default region (if you're using a SageMaker notebook instance, this will be the region where the notebook instance was created). If the repository doesn't exist, the script will create it.

In [49]:
%%sh

# The name of our algorithm
algorithm_name=sagemaker-keras-text-classification

cd ~/SageMaker/amazon-sagemaker-keras-text-classification/container

chmod +x sagemaker_keras_text_classification/train
chmod +x sagemaker_keras_text_classification/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

# On a SageMaker Notebook Instance, the docker daemon may need to be restarted in order
# to detect your network configuration correctly.  (This is a known issue.)
if [ -d "/home/ec2-user/SageMaker" ]; then
  sudo service docker restart
fi

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded
Stopping docker: [  OK  ]
Starting docker:	.[  OK  ]

Step 1/4 : FROM tensorflow-base:1.8.0-cpu-py2
 ---> f0bfaa074d3e
Step 2/4 : ENV PATH="/opt/program:${PATH}"
 ---> Using cache
 ---> c48badb6665c
Step 3/4 : COPY sagemaker_keras_text_classification /opt/program
 ---> f74ebb27711d
Step 4/4 : WORKDIR /opt/program
 ---> Running in 06e4f795d1ed
Removing intermediate container 06e4f795d1ed
 ---> 3435689f7906
Successfully built 3435689f7906
Successfully tagged sagemaker-keras-text-classification:latest
The push refers to repository [741855114961.dkr.ecr.us-east-1.amazonaws.com/sagemaker-keras-text-classification]
70ee6f0fa5a3: Preparing
00b2dbe4c1b6: Preparing
e4b51eac015b: Preparing
f9a193cdb19d: Preparing
fe604d50586c: Preparing
9801f0d644e2: Preparing
19d4850d25d7: Preparing
4c54072a5034: Preparing
49652298c779: Preparing
e15278fcccca: Preparing
739482a9723d: Preparing
9801f0d644e2: Waiting
19d4850d25d7: Waiting
4c54072a5034: Waiting
49652298c779: Waiting
e15278fcccca: W

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



Once you have your container packaged, you can use it to train and serve models. Let's do that with the algorithm we made above.

## Set up the environment

Here we specify a bucket to use and the role that will be used for working with SageMaker.

In [42]:
# S3 prefix
prefix = 'sagemaker-keras-text-classification'

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

## Create the session

The session remembers our connection parameters to SageMaker. We'll use it to perform all of our SageMaker operations.

In [43]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

## Upload the data for training

When training large models with huge amounts of data, you'll typically use big data tools, like Amazon Athena, AWS Glue, or Amazon EMR, to create your data in S3.  

We can use use the tools provided by the SageMaker Python SDK to upload the data to a default bucket. 

In [50]:
WORK_DIRECTORY = '/home/ec2-user/SageMaker/amazon-sagemaker-keras-text-classification/data'

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)
print(data_location)

s3://sagemaker-us-east-1-741855114961/sagemaker-keras-text-classification


## Create an estimator and fit the model

In order to use SageMaker to fit our algorithm, we'll create an `Estimator` that defines how to use the container to to train. This includes the configuration we need to invoke SageMaker training:

* The __container name__. This is constucted as in the shell commands above.
* The __role__. As defined above.
* The __instance count__ which is the number of machines to use for training.
* The __instance type__ which is the type of machine to use for training.
* The __output path__ determines where the model artifact will be written.
* The __session__ is the SageMaker session object that we defined above.

Then we use fit() on the estimator to train against the data that we uploaded above.

In [51]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/sagemaker-keras-text-classification'.format(account, region)

tree = sage.estimator.Estimator(image,
                       role, 1, 'ml.c5.2xlarge',
                       output_path="s3://{}/output".format(sess.default_bucket()),
                       sagemaker_session=sess)

tree.fit(data_location)

2019-06-11 09:30:23 Starting - Starting the training job...
2019-06-11 09:30:26 Starting - Launching requested ML instances......
2019-06-11 09:31:40 Starting - Preparing the instances for training......
2019-06-11 09:32:37 Downloading - Downloading input data
2019-06-11 09:32:37 Training - Downloading the training image...
2019-06-11 09:33:13 Training - Training image download completed. Training in progress....
Instructions for updating:
Colocations handled automatically by placer.
2019-06-11 09:33:36.119620: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2019-06-11 09:33:36.168543: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3000000000 Hz
2019-06-11 09:33:36.170397: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0xb4e0880 executing computations on platform Host. Devices:
2019-06-11 09:33:36.170418: I tensorflow/compiler/xla/service/se

## Deploy the model

Deploying the model to SageMaker hosting just requires a `deploy` call on the fitted model. This call takes an instance count, instance type, and optionally serializer and deserializer functions. These are used when the resulting predictor is created on the endpoint.

In [52]:
from sagemaker.predictor import json_serializer
predictor = tree.deploy(1, 'ml.m5.xlarge', serializer=json_serializer)

---------------------------------------------------------------------------------------------------!

In [53]:
request = { "input": "Deadpool 2 Has More Swearing, Slicing and Dicing from Ryan Reynolds"}

print(predictor.predict(request).decode('utf-8'))

{"result": "Entertainment"}


In [65]:
import json
news_dataset_sampled = news_dataset.sample(frac=0.0001)
for i, n in enumerate(range(news_dataset_sampled.shape[0])):    
    category = news_dataset_sampled.iloc[i][3]
    if category == "b":
        category = "Business"
    elif category == "t":
        category = "Science & Technology"
    elif category == "e":
        category = "Entertainment"
    elif category == "m":
        category = "Health & Medicine"
    else:
        category = "unknown"
    request = {"input": news_dataset_sampled.iloc[i][0]}
    result = json.loads(predictor.predict(request).decode('utf-8'))["result"]
    print("{}. {} - Expected: {}, Predicted: {}".format(n+1, news_dataset_sampled.iloc[i][0], category,result))
    

1. A closer look at Amazon's new Fire smartphone - Expected: Business, Predicted: Science & Technology
2. Lake Forest Businessman Steps in to Save Crumbs - Expected: Business, Predicted: Science & Technology
3. Rob Kardashian Spotted at Gym Amid Sizzurp Concerns; Does He Need Rehab  ... - Expected: Entertainment, Predicted: Entertainment
4. Rite Aid to Administer MMR Vaccines in Ohio - Expected: Health & Medicine, Predicted: Health & Medicine
5. The Linux Foundation Aims To Prevent Future Heartbleed Bugs With Its Core  ... - Expected: Science & Technology, Predicted: Science & Technology
6. Google Earnings Impacted by Nest: What Wall Street's Saying - Expected: Business, Predicted: Science & Technology
7. MacRumors: Apple Close to Acquiring Radio Streaming Service Swell for $30  ... - Expected: Science & Technology, Predicted: Science & Technology
8. Netflix raises prices by a $1 for new subscribers (Update) - Expected: Science & Technology, Predicted: Science & Technology
9. Slowdown 

## Optional cleanup

When you're done with the endpoint, you'll want to clean it up.

In [ ]:
sess.delete_endpoint(predictor.endpoint)